In [6]:
'''
模型融合

数据：设备台账（日期）数据、站场运营（站场巡检）数据、站场运营设备（设备巡检）数据、站场运行（设备告警）数据、站场运行（站点气损）数据
输入：Datafram格式
输出：所有站点的各类分数Score_fram
        设备是否过期 + 设备安装时间 + 设备年检情况 = 台账分数
        站场巡检状态 + 站场巡检次数 + 站场巡检时间 + 设备巡检状态 + 设备巡检次数 = 运营分数
        站场告警分数 + 站场气损分数 = 运行分数
'''

import pandas as pd
import numpy as np
import datetime
import time
import urllib.request
import json
from equipdate import equip_data_score, equip_data_score_calculation
from check import station_check_score
from alerts_loss import alert_score

'''
数据初始化

设备类别权重：
设备权重：

'''
global equip_data
global station_check
global equip_check
global tank_alerts
global now_date           # 当前日期
global expire_deadline    # 设备最多过期多少天 则判为零分
global check_deadline    # 设备最多年检多少天 则判为零分
global check_arrange_day  # 检查的时间范围
global weights_check_a   #巡检状态2的权重
global weights_check_b   #巡检状态3的权重
global standard_station_num     # 标准巡检次数
global standard_station_time     # 标准巡检时长  单位：min
global equip_num # 设备数量
global weights_equip # 设备权重
global standard_equip_num     # 标准设备巡检次数
global alertsType_weight    #告警类型权重
global illegal_score   #非法的分数 比如日期都一样的 或者缺失数据 都算非法
global alert_1       # 告警类型1 一般 时的扣分
global alert_2       # 告警类型2 
global alert_3       # 告警类型3 严重 时的扣分
alert_1 = 1
alert_2 = 10
alert_3 = 100
# alertsType_weight = ？？？
illegal_score = 0  # 暂时设为0分
standard_equip_num = [3 for i in range(11)]
weights_equip = [0,0,0,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0.1]
equip_num = 11
weights_check_a = 0.8
weights_check_b = 0.2
now_date = datetime.date.today()
check_arrange_day = 365
expire_deadline = 30
check_deadline = 30
standard_station_time = 10
standard_station_num = 5


'''
预定义数据来源地址
'''
## 设备台账（日期）数据
filename_equip_data = 'D:/document/gas_healthy/data/site_ledgers_20180710.xlsx'
## 站场运营（站场巡检）数据
filename_station_check = 'D:/document/gas_healthy/data/tank_inspects_20180710.xlsx'
## 站场运营（设备巡检）数据
filename_equip_check = 'D:/document/gas_healthy/data/inspect_items_20180714.xlsx'
## 站场运行（站点告警）数据
filename_tank_alerts = 'D:/document/gas_healthy/data/tank_alerts_20180710.xlsx'
## 站场运行（站点气损）数据
filename_gasloss = 'D:/document/gas_healthy/data/rpt_daily_site.xlsx'

'''
获得数据

设备台账（日期）数据：equip_data
站场运营（站场巡检）数据：station_check
站场运营（设备巡检）数据：equip_check
站场运行（告警气损）数据：tank_alerts,gasloss_data
'''
equip_data = pd.read_excel(filename_equip_data)
equip_data = equip_data[equip_data['state'] == 1]
station_check = pd.read_excel(filename_station_check)
equip_check = pd.read_excel(filename_equip_check)
tank_alerts = pd.read_excel(filename_tank_alerts)
gasloss_data = pd.read_excel(filename_gasloss)




In [8]:

def score_cal(station_id):
    Score_fram = pd.DataFrame(index=station_id)
    station_expire_score = np.zeros(len(station_id)) #过期
    station_set_score = np.zeros(len(station_id))    #安装
    station_yearcheck_score = np.zeros(len(station_id))  #年检
    station_state_score = np.zeros(len(station_id))  #站点巡检状态
    station_time_score = np.zeros(len(station_id))   #站点巡检时间
    station_num_score = np.zeros(len(station_id))    #站点巡检次数
    equip_score = np.zeros(len(station_id))          #设备巡检状态
    equip_total_num_score = np.zeros(len(station_id)) #设备巡检次数
    station_alerts_score = np.zeros(len(station_id))
    for i in range(len(station_id)):
        equip_data_fusion = equip_data_score(station_id[i],equip_data,now_date,check_deadline,illegal_score,expire_deadline)
        station_expire_score[i],station_set_score[i],station_yearcheck_score[i] = equip_data_score_calculation(equip_data_fusion,expire_deadline,check_deadline,illegal_score)
        station_state_score[i],station_time_score[i],station_num_score[i],equip_score[i],equip_total_num_score[i] = station_check_score(station_id[i],equip_num,station_check,now_date,check_arrange_day,equip_check,standard_equip_num,weights_equip,standard_station_time,weights_check_a,weights_check_b)
        station_alerts_score[i] = alert_score(station_id[i],equip_data,tank_alerts,now_date,check_arrange_day,alert_1,alert_2,alert_3)
    Score_fram['expire_score'] = station_expire_score
    Score_fram['set_score'] = station_set_score
    Score_fram['yearcheck_score'] = station_yearcheck_score
    Score_fram['station_state_score'] = station_state_score
    Score_fram['station_time_score'] = station_time_score
    Score_fram['station_num_score'] = station_num_score
    Score_fram['equip_state_score'] = equip_score
    Score_fram['equip_num_score'] = equip_total_num_score
    Score_fram['station_alerts_score'] = station_alerts_score
    Score_fram['taizhang_score'] = (Score_fram['expire_score'] + Score_fram['set_score'] + Score_fram['yearcheck_score'])/3
    Score_fram['yunying_score'] = (Score_fram['station_state_score'] + Score_fram['station_time_score'] + Score_fram['station_num_score'] 
                                   + Score_fram['equip_state_score'] + Score_fram['equip_num_score'])/5
    Score_fram['yunxing_score'] = Score_fram['station_alerts_score'] # 还要加气损
    Score_fram['total_score'] = Score_fram['taizhang_score']*0.2 + Score_fram['yunying_score']*0.5 + Score_fram['yunxing_score']*0.3
    return Score_fram

if __name__ == "__main__":
    ### 得到所有站点编号 共108个站点
    station_id = equip_data['site_id'].unique() 
    score = score_cal(station_id)
    print(score)

TypeError: equip_data_score_calculation() takes 3 positional arguments but 4 were given